In [4]:
%gui qt
%matplotlib qt
import xmcd_projection
from xmcd_projection import deep_reload
deep_reload(xmcd_projection)
from xmcd_projection import *
from glob import glob
import os

In [5]:
folder_path = r"F:\PhD_data\Simulations\Alba_spirals\scaled\al6-1-1_asbuiltnotop_scaled"

data_folder = os.path.join(folder_path, "data")
mag_files = glob(os.path.join(data_folder, "*.csv"))
msh_file = glob(os.path.join(data_folder, "*.msh"))[0]

# sort mag files
indx = [int(mf.split('.')[-2]) for mf in mag_files]
# indx = [int(mf.split('_')[-1].split('.')[0]) for mf in mag_files]
mag_files = [mf for _, mf in sorted(zip(indx, mag_files))]
# get the mesh and magnetisation
mesh = Mesh.from_file(msh_file)

In [6]:
from xmcd_projection.projection import project_structure
p = get_projection_vector(90, 15)

struct = mesh.get_bounding_struct()
struct_projected = project_structure(struct, p)

In [15]:
vis = MeshVisualizer(struct, struct_projected)
vis.show()

In [ ]:

index_triangle, index_ray, locations = ray_triangle_id(
    triangles, ray_origins, ray_directions)

In [ ]:
piercings_list = get_piercings(mesh, p)

In [7]:

from trimesh.ray.ray_triangle import ray_triangle_id
from time import time

ray_origins = struct_projected.triangles_center
triangles = mesh.triangles
n = ray_origins.shape[0]
ray_directions = np.repeat(p[None, :], n, axis=0)
t0 = time()
index_triangle, index_ray, locations = ray_triangle_id(
    triangles, ray_origins, ray_directions)
time()-t0

In [11]:
from tqdm import tqdm
import trimesh
from trimesh.ray.ray_triangle import RayMeshIntersector
from xmcd_projection.raytracing import get_piercings_frompt_lengths
from trimesh.ray.ray_triangle import ray_triangle_id
from trimesh import triangles as triangles_mod
from joblib import Parallel, delayed
from time import time
t0 = time()

ray_origins = struct_projected.triangles_center
triangles = mesh.triangles
triangles_normal = triangles_mod.normals(triangles)[0]
tree = triangles_mod.bounds_tree(triangles)

pnew = p[np.newaxis, :]

def ray_id_fun(orig): return ray_triangle_id(
    triangles, orig[np.newaxis, :], pnew, triangles_normal=triangles_normal, tree=tree)
ray_ids_generator = (ray_id_fun(orig) for orig in ray_origins)

# ray_ids_generator = Parallel(n_jobs=5)(delayed(ray_id_fun)(orig) for orig in ray_origins)

piercings_list = [
    get_piercings_frompt_lengths(
        ri[2],
        ri[0] // 4)
    for ri in ray_ids_generator]
# ray_id_fun = lambda tri, orig, dr: ray_triangle_id(tri, orig[np.newaxis, :], dr[np.newaxis, :], triangles_normal=triangles_normal, tree=tree)
# ray_ids = Parallel(n_jobs=5, prefer='threads')(delayed(ray_id_fun)(triangles, ray_origins[i, :], p) for i in range(ray_origins.shape[0]))
# for i in tqdm(range(ray_origins.shape[0])):
#     ray_ids.append(raymesh.intersects_id(ray_origins[i:i+1,:], ray_directions[i:i+1,:], return_locations=True))
    
# piercings_list = [
#         get_piercings_frompt_lengths(
#             ri[2],
#             ri[0]//4)
#         for ri in ray_ids]
#         ray_ids.append(ray_triangle_id(
#     triangles, ray_origins[i:i+1,:], ray_directions[i:i+1,:])

# ray_ids = [ray_triangle_id(
#     triangles, ray_origins[i:i+1,:], ray_directions[i:i+1,:]) for i in range(ray_origins.shape[0])]
time() - t0

4.020192384719849

In [13]:
piercings_list[0]

(array([1.57584545e+00, 2.15633670e-01, 1.36170550e+00, 1.11690918e+00,
        3.67643037e+00, 8.21393687e-01, 2.83037785e+00, 1.19788373e-01,
        4.98082614e-02, 1.83106625e-01, 8.25954384e-02, 3.69264813e+00,
        1.80565156e+00, 5.21786108e+00, 3.24528079e-03, 1.57895597e+00,
        1.58428549e+00, 4.47678950e-01, 3.00885698e-01]),
 array([   1,   16,  239,  240,  728, 1077, 1257, 1923, 1967, 3075, 3146,
        3195, 3227, 3262, 3344, 7210, 7211, 7212, 7213], dtype=int64))

In [34]:
np.sort(ray_ids[100][0]//4)

array([ 109,  109,  152,  152,  156,  156,  204,  220,  229,  486, 1314,
       1355, 2421, 2453, 2453, 3807, 4926, 4926, 7249, 9239], dtype=int64)

In [20]:

deep_reload(xmcd_projection)
from xmcd_projection import *
from xmcd_projection.piercings_utils import get_piercings_frompt_lengths
piercings_list = [
    get_piercings_frompt_lengths(
        locations[index_ray == i],
        index_triangle[index_ray == i] // 4)
    for i in range(n)]

In [21]:
ray_origins = projected_struct.triangles_center
ray_directions = np.repeat(p[None, :], ray_origins.shape[0], axis=0)
triangles = msh.triangles

In [22]:
index_triangle, index_ray, locations = ray_triangle_id(triangles, ray_origins, ray_directions)

In [24]:
index_triangle[index_ray == 90]

array([2290,  601, 1410, 2291, 3196, 3198, 1408, 5171,  448, 5170, 1405,
        602, 1406,  773,  449,  124,  126,  675,  775,  674], dtype=int64)

In [26]:
intersected_tetrahedra_indx

array([   0,    1,    5,   11,   13,   16,   35,   41,   58,   81,  337,
        344,  788, 7336], dtype=int64)

In [34]:
loc_pt = locations[index_ray == 0]
intersected_tetrahedra_indx = index_triangle[index_ray == 0]//4
intersected_tetrahedra_indx_unique = np.unique(intersected_tetrahedra_indx)
intersected_tetrahedra_lengths = np.zeros(intersected_tetrahedra_indx_unique.size)
for i, idx in enumerate(intersected_tetrahedra_indx_unique):
    pts = loc_pt[intersected_tetrahedra_indx == idx]
    intersected_tetrahedra_lengths[i] = np.linalg.norm(pts[0,:] - pts[1,:])
# intersected_tetrahedra_lengths = [np.linalg.norm(intersected_tetrahedra_indx==i)]

In [35]:
intersected_tetrahedra_lengths

array([0.76836384, 1.75096416, 0.07091829, 3.23921025, 0.31088248,
       0.88539146, 0.35712067, 0.57101625, 1.33860802, 0.79454974,
       3.19913954, 0.46852175, 1.31585321, 2.0398098 ])

In [14]:
odd = np.array([len(index_triangle[index_ray == i]) % 2 != 0 for i in range(ray_origins.shape[0])])
np.nonzero(odd)

(array([], dtype=int64),)

In [14]:
np.min(triangles.reshape(-1, 3)[:,2])

1889.1931147895937

In [37]:
index_triangle[:4]

array([67, 54, 52,  0], dtype=int64)

In [39]:
tri0 = index_triangle[index_ray == 0]

In [58]:
tri = np.sort(index_triangle[index_ray == 1260])
# check if the triangles form tetrahedra
diff_tri = np.diff(tri)
keep_indices = 


In [172]:
ray_triangle_id(triangles, ray_origins[1260, :][np.newaxis, :], ray_directions[1260, :][np.newaxis, :])

(array([17175,  9564,  9565, 21153, 17172], dtype=int64),
 array([0, 0, 0, 0, 0], dtype=int64),
 array([[1.09496123e+02, 3.96987236e+00, 5.12740362e-01],
        [1.09496123e+02, 3.96987236e+00, 5.12740362e-01],
        [1.09496123e+02, 6.00758754e+00, 1.05874450e+00],
        [1.09496123e+02, 2.08533247e+00, 7.77941917e-03],
        [1.09496123e+02, 2.08533247e+00, 7.77941917e-03]]))

In [147]:
i = 1260
k = 21153

In [161]:
possible_triangles =triangles[(k//4)*4:(k//4)*4+4, :, :]
ray_triangle_id(possible_triangles, ray_origins[1260, :][np.newaxis, :], ray_directions[1260, :][np.newaxis, :])

(array([1], dtype=int64),
 array([0], dtype=int64),
 array([[1.09496123e+02, 2.08533247e+00, 7.77941917e-03]]))

In [ ]:
pt = ray_origins[1260, :][np.newaxis, :]
points_on_planes = triangle_ray_intersection(p, pt, possible_triangles)

possible_triangles_pierced_indx = origin_in_triangle(
    possible_triangles - points_on_planes[:, np.newaxis, :])

In [166]:
possible_triangles_pierced_indx

array([False,  True, False,  True])

In [156]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Create a new plot
fig = plt.figure()
ax = fig.add_subplot((111), projection='3d', proj_type='ortho')

selected_triangles = triangles[(k//4)*4+1, :, :]
vts = selected_triangles.reshape(-1, 3)
ax.plot_trisurf(vts[:,0], vts[:,1], vts[:,2])
ax.scatter(vts[:,0], vts[:,1], vts[:,2], 'ko')
ax.set_xlabel('x')

pt0 = ray_origins[i, :]
nx0 = np.cross(pt0, p)
y = np.linspace(vts[:,1].min(), vts[:,1].max(), 100)
# y = np.linspace(points_on_planes[:,1].min(), points_on_planes[:,1].max(), 100)
x = (y*p[0]+nx0[2])/p[1]
z = (y*p[2]/p[1]-nx0[0])/p[1]

ax.plot(x, y, z, c='red')


In [157]:
pt0 = locations[index_ray == i][4, :]
ax.scatter(pt0[0], pt0[1], pt0[2], c='yellow', s=1000)

In [120]:
locations

array([[111.91509843,  22.08599224,  15.62172815],
       [111.91509843,  22.08599224,  15.62172815],
       [111.91509843,  22.38628166,  15.70219046],
       ...,
       [ -4.39426193, -95.9793848 , 442.39283833],
       [ -4.39426193, -94.09961778, 442.89652038],
       [ -4.39426193, -94.09961778, 442.89652038]])

In [61]:
keep_indices = np.where(diff_tri<=3)
keep_indices = 

(array([0, 2], dtype=int64),)

In [54]:
np.diff(wrong_tri)

array([    1,   488,     1,   651,     2,   326,     2,   110,     2,
        1285,     1, 14211,     1,  4176], dtype=int64)

In [49]:
odd = np.array([len(index_triangle[index_ray == i]) % 2 != 0 for i in range(ray_origins.shape[0])])
np.nonzero(odd)

(array([1095, 1260, 1416, 1509, 1562, 1823, 2169, 2213, 2349, 2406, 2557,
        2911, 3705, 4567, 4727, 4905, 4918, 5072, 5176, 5549], dtype=int64),)

In [ ]:
tetra_wrong = 

In [42]:
np.sort(tri0)

array([    0,     2,     5,     7,    22,    23,    44,    46,    52,
          54,    65,    67,   142,   143,   165,   166,   234,   235,
         325,   326,  1348,  1349,  1376,  1379,  3152,  3153, 29345,
       29346], dtype=int64)

In [34]:
mesh.triangles

array([[[115.07399134,  25.58664678,  16.26597271],
        [116.34854464,  21.08865516,  21.4879623 ],
        [110.80483705,  22.32861196,  14.58337199]],

       [[111.19838654,  22.15640576,  23.50294614],
        [116.34854464,  21.08865516,  21.4879623 ],
        [110.80483705,  22.32861196,  14.58337199]],

       [[111.19838654,  22.15640576,  23.50294614],
        [115.07399134,  25.58664678,  16.26597271],
        [110.80483705,  22.32861196,  14.58337199]],

       ...,

       [[ -2.29675924, -92.49135962, 444.72822183],
        [ -6.06352082, -89.76801016, 447.38076042],
        [  0.84311385, -89.43834269, 444.52670159]],

       [[ -2.29675924, -92.49135962, 444.72822183],
        [ -1.738934  , -88.66054527, 441.8358262 ],
        [  0.84311385, -89.43834269, 444.52670159]],

       [[ -2.29675924, -92.49135962, 444.72822183],
        [ -1.738934  , -88.66054527, 441.8358262 ],
        [ -6.06352082, -89.76801016, 447.38076042]]])